In [ ]:
import importlib
import local_utils; importlib.reload(local_utils)
from local_utils import *

In [ ]:
global_start = time()

In [ ]:
train_df = pd.read_csv(DATA_DIR + '/train_v2.csv')
test_df = pd.read_csv(DATA_DIR + '/sample_submission_v2.csv')

label_map, inv_label_map, Y = process_labels(train_df)
print("Shape of Y {}".format(Y.shape))

train_folds, valid_folds = stratified_kfold_sampling(Y, n_splits=10, random_state=1000)
print("Shape of train folds {}".format(train_folds.shape))
print("Shape of valid folds {}".format(valid_folds.shape))

image_size = 192

In [ ]:
model_name = "deep_model_192x192"

fold_inx = 1
batch_size = 32
all_steps = np.ceil(len(train_df)/batch_size)
train_steps = np.ceil(len(train_folds[fold_inx])/batch_size)
valid_steps = np.ceil(len(valid_folds[fold_inx])/batch_size)
test_steps = np.ceil(len(test_df)/batch_size)

In [ ]:
# cache_images(DATA_DIR+"/train_images_"+str(image_size)+".dat", train_df['image_name'].values[train_folds[fold_inx]], DATA_DIR+'/train-jpg', True, image_size, image_size, dtype=np.float32)
# cache_images(DATA_DIR+"/valid_images_"+str(image_size)+".dat", train_df['image_name'].values[valid_folds[fold_inx]], DATA_DIR+'/train-jpg', True, image_size, image_size, dtype=np.float32)
# cache_images(DATA_DIR+"/test_images_"+str(image_size)+".dat", test_df['image_name'].values, DATA_DIR+'/test-jpg', True, image_size, image_size, dtype=np.float32)

In [ ]:
train_array = load_carray(DATA_DIR+'/train_images_'+str(image_size)+'.dat')
valid_array = load_carray(DATA_DIR+'/valid_images_'+str(image_size)+'.dat')

In [ ]:
x_input = Input(shape=(image_size, image_size, 3))

x = BatchNormalization()(x_input)
x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
xm = MaxPooling2D()(x)
xa = AveragePooling2D()(x)

xm = Conv2D(64, (3, 3), padding='same', activation='relu')(xm)
xm = Conv2D(64, (3, 3), activation='relu')(xm)
xm = BatchNormalization()(xm)
xm = MaxPooling2D()(xm)

xm = Conv2D(128, (3, 3), padding='same', activation='relu')(xm)
xm = Conv2D(128, (3, 3), activation='relu')(xm)
xm = BatchNormalization()(xm)
xm = MaxPooling2D()(xm)

xm = Conv2D(256, (3, 3), padding='same', activation='relu')(xm)
xm = Conv2D(256, (3, 3), activation='relu')(xm)
xm = BatchNormalization()(xm)
xm = MaxPooling2D()(xm)

xm = Conv2D(256, (3, 3), padding='same', activation='relu')(xm)
xm = Conv2D(256, (3, 3), activation='relu')(xm)
xm = BatchNormalization()(xm)

xa = Conv2D(64, (3, 3), padding='same', activation='relu')(xa)
xa = Conv2D(64, (3, 3), activation='relu')(xa)
xa = BatchNormalization()(xa)
xa = AveragePooling2D()(xa)

xa = Conv2D(128, (3, 3), padding='same', activation='relu')(xa)
xa = Conv2D(128, (3, 3), activation='relu')(xa)
xa = BatchNormalization()(xa)
xa = AveragePooling2D()(xa)

xa = Conv2D(256, (3, 3), padding='same', activation='relu')(xa)
xa = Conv2D(256, (3, 3), activation='relu')(xa)
xa = BatchNormalization()(xa)
xa = AveragePooling2D()(xa)

xa = Conv2D(256, (3, 3), padding='same', activation='relu')(xa)
xa = Conv2D(256, (3, 3), activation='relu')(xa)
xa = BatchNormalization()(xa)

xm = GlobalMaxPooling2D()(xm)
xa = GlobalAveragePooling2D()(xa)
x = concatenate([xm, xa])

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

x_output = Dense(17, activation='sigmoid')(x)

In [ ]:
model = Model(inputs=x_input, outputs=x_output)

# model_128x128 = load_model(DATA_DIR+'/models/deep_model_1_backup.h5', compile=False)
# model.set_weights(model_128x128.get_weights())

# model_256x256 = load_model(DATA_DIR+'/models/deep_model_256x256_1_backup.h5', compile=False)
# model.set_weights(model_256x256.get_weights())

model.summary()

In [ ]:
def getImageDataGenerator():
    return ImageDataGenerator(
                rotation_range=90,
                horizontal_flip=True,
                vertical_flip=True,)

all_gen = getImageDataGenerator()
train_gen = getImageDataGenerator()
valid_gen = getImageDataGenerator()
test_gen = getImageDataGenerator()

In [ ]:
f2_history = F2History(valid_array, Y[valid_folds[fold_inx]], 
                       valid_gen, valid_steps, batch_size)

model_checkpoint = ModelCheckpoint(
                DATA_DIR+'/models/'+model_name+'_'+str(fold_inx)+'.h5', 
                monitor='val_fn_loss', 
                verbose=1, 
                save_best_only=True, 
                save_weights_only=False, 
                mode='min',
                period=1)

In [ ]:
# del model
model = load_model(DATA_DIR+'/models/'+model_name+'_'+str(fold_inx)+'.h5', compile=False)

In [ ]:
# opt=optimizers.Adam(lr=1e-2); epochs=4;
opt=optimizers.Adam(lr=1e-3); epochs=2;
# opt=optimizers.Adam(lr=1e-4, decay=2e-5); epochs=5;
# opt=optimizers.Adam(lr=1e-5, decay=33e-6); epochs=3;
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[fn_loss, f2, 'accuracy'])

In [ ]:
# model.evaluate_generator(
#     valid_gen.flow(valid_array, Y[valid_folds[fold_inx]], batch_size=batch_size, shuffle=False),
#     steps=valid_steps)

In [ ]:
model.fit_generator(
    generator=train_gen.flow(train_array, Y[train_folds[fold_inx]], batch_size=batch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    validation_data=valid_gen.flow(valid_array, Y[valid_folds[fold_inx]], batch_size=batch_size, shuffle=False),
    validation_steps=valid_steps,
    callbacks=[model_checkpoint, f2_history])

In [ ]:
print(f2_history.f2_02_scores)
plt.plot(f2_history.f2_02_scores)
plt.show()

In [ ]:
print(f2_history.f2_05_scores)
plt.plot(f2_history.f2_05_scores)
plt.show()

In [ ]:
stop